In [1]:
import os
import sys
import datetime
import pandas as pd
import boto3

# Inicializa ambiente
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from tabulate import tabulate
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from boto3.dynamodb.conditions import Key
from dotenv import load_dotenv

In [3]:
# Carrega variáveis de ambiente do arquivo .env
load_dotenv()

True

In [4]:
spark = SparkSession.builder \
    .appName("TarefasAbandonadas6meses") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [5]:
dynamodb = boto3.resource('dynamodb', region_name='sa-east-1')
table = dynamodb.Table("ListaMercado")

In [6]:
hoje = datetime.date.today()

In [7]:
def gerar_datas_6_meses(data_atual):
    datas = []
    data_inicio = data_atual - datetime.timedelta(days=180)
    data_iter = data_inicio
    while data_iter <= data_atual:
        datas.append(data_iter.strftime("%Y%m%d"))
        data_iter += datetime.timedelta(days=1)
    return datas


In [8]:
todas_datas = gerar_datas_6_meses(hoje)

In [9]:
todos_registros = []
datas_com_dados = []
total_queries = 0
lote_size = 50

In [10]:
for i in range(0, len(todas_datas), lote_size):
    lote_datas = todas_datas[i:i+lote_size]
    for data in lote_datas:
        pk = f"LIST#{data}"
        total_queries += 1
        try:
            response = table.query(KeyConditionExpression=Key("PK").eq(pk))
            registros = response.get("Items", [])
            if registros:
                todos_registros.extend(registros)
                datas_com_dados.append(data)
        except:
            pass

if not todos_registros:
    print("Nenhum registro encontrado!")
    spark.stop()
    exit()

print(f"- Total de queries executadas: {total_queries}")
print(f"- Datas com dados: {len(datas_com_dados)}")
print(f"- Total de registros encontrados: {len(todos_registros)}")

- Total de queries executadas: 181
- Datas com dados: 132
- Total de registros encontrados: 242


In [11]:
df_pandas = pd.DataFrame(todos_registros)
df_em_aberto = df_pandas[df_pandas['status'] == 'todo'].copy()
count_em_aberto = len(df_em_aberto)

if count_em_aberto == 0:
    print("Nenhuma tarefa em aberto encontrada!")
    spark.stop()
    exit()

# Dias em aberto
hoje_pd = pd.Timestamp(hoje)
df_em_aberto['date'] = pd.to_datetime(df_em_aberto['date'])
df_em_aberto['dias_aberto'] = (hoje_pd - df_em_aberto['date']).dt.days

# Critérios de abandono
tarefas_abandonadas_normais = df_em_aberto[
    (df_em_aberto['taskType'] == 'Tarefa a Ser Feita') & (df_em_aberto['dias_aberto'] > 15)
]
itens_abandonados = df_em_aberto[
    (df_em_aberto['taskType'] == 'Item de Compra') & (df_em_aberto['dias_aberto'] > 30)
]

df_abandonadas = pd.concat([tarefas_abandonadas_normais, itens_abandonados], ignore_index=True)
count_abandonadas = len(df_abandonadas)

# Relatório por mês
df_abandonadas['mes_criacao'] = df_abandonadas['date'].dt.strftime('%Y-%m')
ultimos6_meses = [(hoje.replace(day=1) - datetime.timedelta(days=30*i)).strftime("%Y-%m") for i in range(6)]

resultado_final = []
for mes in ultimos6_meses:
    tarefas_mes = df_abandonadas[df_abandonadas['mes_criacao'] == mes]
    resultado_final.append({
        "mes": mes,
        "Tarefa a Ser Feita": len(tarefas_mes[tarefas_mes['taskType'] == 'Tarefa a Ser Feita']),
        "Item de Compra": len(tarefas_mes[tarefas_mes['taskType'] == 'Item de Compra'])
    })

df_relatorio = pd.DataFrame(resultado_final).set_index('mes').T

# Salvar
df_relatorio.to_csv("../data/relatorioAbandono.csv")
df_relatorio.to_excel("../data/relatorioAbandono.xlsx")

# Exibir tabela simulada do Excel no terminal
print("\n=== TABELA DE RELATÓRIO ===")
tabela_formatada = tabulate(df_relatorio, headers="keys", tablefmt="grid", stralign="center", numalign="center")
print(tabela_formatada)

# Resumo
print(f"- Tarefas em aberto analisadas: {count_em_aberto}")
print(f"- Tarefas abandonadas encontradas: {count_abandonadas}")
print(f"\n Análise concluída!")

spark.stop()



=== TABELA DE RELATÓRIO ===
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|                    |  2025-06  |  2025-05  |  2025-04  |  2025-03  |  2025-02  |  2025-01  |
+====================+===========+===========+===========+===========+===========+===========+
| Tarefa a Ser Feita |     0     |    11     |    13     |    12     |    19     |     7     |
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Item de Compra   |     0     |     3     |     7     |     6     |     4     |     8     |
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
- Tarefas em aberto analisadas: 117
- Tarefas abandonadas encontradas: 108

 Análise concluída!
